In [26]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"  # ajuste conforme sua instalação
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["SPARK_LOCAL_IP"] = "127.0.0.1"

import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum, explode, collect_list
import google.protobuf 
print("Versão do PySpark",ps.__version__)

Versão do PySpark 4.0.1


In [27]:
#iniciar sesssão quando se trabalho com spark é necessário abrir uma sessão
spark = SparkSession.builder.appName("Demo").getOrCreate()

#Leitura do arquivo CSV
vendas = spark.read.csv("./vendas.csv", header=True, inferSchema=True)

#leitura de clientesSimples.json
clientes = spark.read.json("./clientesSimples.json")

#leitura do arquivo JSON
#df_raw = spark.read.option("multiLine", True).json("./clientesSimples.json")
#df_raw.printSchema()   # deve mostrar coluna 'clientes' como array<struct>

# Explodir o array para ter um registro por cliente
#clientes = df_raw.select(explode(col("clientes")).alias("cliente")).select("cliente.*")

# Remover eventuais registros corrompidos
#if "_corrupt_record" in clientes.columns:
#    clientes = clientes.filter(col("_corrupt_record").isNull()).drop("_corrupt_record")
    

In [13]:
#Exibir o esquema dos DataFrames
print("Esquenma do DataFrame de vendas")
vendas.printSchema()

#Exibir o esquema dos DataFrames
print("\n Esquema do DataFrame de clientes")
clientes.printSchema()

#Exibição dos daos
print("Dados de vendas:")
vendas.show()

print("\n Dados de clientes:")
# Mostrar sem truncar (use truncate=False ou um inteiro)
clientes.show()




Esquenma do DataFrame de vendas
root
 |-- ID_Venda: integer (nullable = true)
 |-- Data: date (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Categoria: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Preco_Unitario: integer (nullable = true)
 |-- Valor_Total: integer (nullable = true)
 |-- Forma_Pagamento: string (nullable = true)
 |-- Vendedor: string (nullable = true)
 |-- ID_Cliente: integer (nullable = true)


 Esquema do DataFrame de clientes
root
 |-- ativo: boolean (nullable = true)
 |-- categoria_favorita: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- email: string (nullable = true)
 |-- id: long (nullable = true)
 |-- nome: string (nullable = true)
 |-- total_compras: long (nullable = true)
 |-- ultima_compra: string (nullable = true)

Dados de vendas:
+--------+----------+--------------------+-----------+----------+--------------+-----------+---------------+--------------+----------+
|ID_Venda|      Data|          

In [14]:
#filtragem
vendas_filtradas = vendas.filter(col("quantidade") >= 3)
print("Vendas com quantidade maior ou igual a 3:")
vendas_filtradas.show()

print("\n Vendas Menor do que 3:")
vendas_filtradas = vendas.filter(col("quantidade") < 3)
vendas_filtradas.show()

#Filtrar clientes de São Paulo
#clientes_filtrados = clientes.filter((col("clientes.cidade").alias("cliente.cidade") == "São Paulo"))
print("\n Clientes de São Paulo")

clientes_filtrados = clientes.filter(col("cidade") == "São Paulo")
clientes_filtrados.show()


Vendas com quantidade maior ou igual a 3:
+--------+----------+--------------------+-----------+----------+--------------+-----------+---------------+--------------+----------+
|ID_Venda|      Data|             Produto|  Categoria|Quantidade|Preco_Unitario|Valor_Total|Forma_Pagamento|      Vendedor|ID_Cliente|
+--------+----------+--------------------+-----------+----------+--------------+-----------+---------------+--------------+----------+
|     997|2024-11-05|            Mousepad| Acessórios|         4|            50|        200|        Crédito|  Maria Santos|      7988|
|     998|2024-11-05|             Hub USB| Acessórios|         4|            80|        320|            PIX|     Ana Costa|      4182|
|     995|2024-11-06|     Memória RAM 8GB|   Hardware|         3|           200|        600|            PIX|     Ana Costa|      6360|
|     993|2024-11-07|         Headset 7.1|Periféricos|         5|           250|       1250|        Crédito|     Ana Costa|      4847|
|     992|202

In [16]:
#Agregação: total de vendas por produto
valor_total = vendas.agg(sum("Valor_Total").alias("Valor_Total_Vendas"))
print("Valor total das vendas:")
valor_total.show()

Valor total das vendas:


+------------------+
|Valor_Total_Vendas|
+------------------+
|           1767005|
+------------------+



In [ ]:
#média
idade_media = clientes.agg(avg("idade").alias("Idade_Media"))
print("Idade média dos clientes:")
idade_media.show()

Idade média dos clientes:
+-----------+
|Idade_Media|
+-----------+
|       34.4|
+-----------+



In [33]:
#união
#criand um novo dataframe de vendas adicionais
vendas_adicionais = spark.createDataFrame([
    (1000,'2024-11-04',"Cabo HDMI","Acessórios",2,45,90,"Débito","João Silva",3295),
    (1000,'2024-11-04',"Cabo HDMI","Acessórios",2,45,90,"Débito","João Silva Pereira",3296)
    ],["ID_Venda","Data","Produto","Categoria","Quantidade","Preco_Unitario","Valor_Total","Forma_Pagamento","Vendedor","ID_Cliente"])

uniao_vendas = vendas.union(vendas_adicionais)
print("Vendas após a união com vendas adicionais:")
uniao_vendas.show()

#escrever o vendas.csv
uniao_vendas.write.csv("./vendas_atualizadas.csv", header=True, mode="overwrite")

#escrever oa cleientes
clientes.write.json("./clientes_atualizados.json", mode="overwrite")

spark.stop()

Vendas após a união com vendas adicionais:
+--------+----------+--------------------+-----------+----------+--------------+-----------+---------------+--------------+----------+
|ID_Venda|      Data|             Produto|  Categoria|Quantidade|Preco_Unitario|Valor_Total|Forma_Pagamento|      Vendedor|ID_Cliente|
+--------+----------+--------------------+-----------+----------+--------------+-----------+---------------+--------------+----------+
|    1000|2024-11-04|           Cabo HDMI| Acessórios|         2|            45|         90|         Débito|    João Silva|      3294|
|     999|2024-11-04|      Adaptador HDMI| Acessórios|         2|            35|         70|         Boleto|Pedro Oliveira|      7956|
|     997|2024-11-05|            Mousepad| Acessórios|         4|            50|        200|        Crédito|  Maria Santos|      7988|
|     998|2024-11-05|             Hub USB| Acessórios|         4|            80|        320|            PIX|     Ana Costa|      4182|
|     995|20